<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
from tensorflow.keras.callbacks import LambdaCallback, EarlyStopping, TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

import numpy as np
import random
import sys
import requests
import os
import datetime

In [2]:
url = "https://www.gutenberg.org/files/100/100-0.txt"

r = requests.get(url)

In [3]:
r.encoding = r.apparent_encoding

data = r.text

In [4]:
data[:100]

'\r\nProject Gutenberg’s The Complete Works of William Shakespeare, by William\r\nShakespeare\r\n\r\nThis eBo'

In [5]:
data = data.split('\r\n')

In [6]:
# Skip the Table of Contents
data = data[135:]

In [7]:
toc = [l.strip() for l in data[44:130:2]]

In [8]:
toc

['For where is she so fair whose uneared womb',
 'Or who is he so fond will be the tomb',
 'Thou art thy mother’s glass and she in thee',
 'So thou through windows of thine age shalt see,',
 'But if thou live remembered not to be,',
 '',
 '4',
 'Unthrifty loveliness why dost thou spend,',
 'Nature’s bequest gives nothing but doth lend,',
 'Then beauteous niggard why dost thou abuse,',
 'Profitless usurer why dost thou use',
 'For having traffic with thy self alone,',
 'Then how when nature calls thee to be gone,',
 'Thy unused beauty must be tombed with thee,',
 '',
 '5',
 'Those hours that with gentle work did frame',
 'Will play the tyrants to the very same,',
 'For never-resting time leads summer on',
 'Sap checked with frost and lusty leaves quite gone,',
 'Then were not summer’s distillation left',
 'Beauty’s effect with beauty were bereft,',
 'But flowers distilled though they with winter meet,',
 '',
 '6',
 'Then let not winter’s ragged hand deface,',
 'Make sweet some vial; tre

In [9]:
locations = {id_:{'title':title, 'start':-99} for id_,title in enumerate(toc)}

In [10]:
# Fixing Titles
locations[9]['title'] = 'THE LIFE OF KING HENRY V'

In [11]:
# Start 
for e,i in enumerate(data):
    for t,title in enumerate(toc):
        if title in i:
            locations[t].update({'start':e})
            
# End            
for title in toc:
    
    t = 0
    
    while t < len(toc):
        print(t)
        end = locations[t+1]['start'] - 1
        locations[t]['end'] = end
        t += 1

    # Last One
    locations[t]['end'] = len(data)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


KeyError: 43

In [16]:
locations[9]

{'title': 'THE LIFE OF KING HENRY V', 'start': -99, 'end': 47572}

In [17]:
locations

{0: {'title': 'ALL’S WELL THAT ENDS WELL', 'start': 2777, 'end': 7738},
 1: {'title': 'THE TRAGEDY OF ANTONY AND CLEOPATRA',
  'start': 7739,
  'end': 11840},
 2: {'title': 'AS YOU LIKE IT', 'start': 11841, 'end': 14631},
 3: {'title': 'THE COMEDY OF ERRORS', 'start': 14632, 'end': 17832},
 4: {'title': 'THE TRAGEDY OF CORIOLANUS', 'start': 17833, 'end': 27806},
 5: {'title': 'CYMBELINE', 'start': 27807, 'end': 27824},
 6: {'title': 'THE TRAGEDY OF HAMLET, PRINCE OF DENMARK',
  'start': 27825,
  'end': 34511},
 7: {'title': 'THE FIRST PART OF KING HENRY THE FOURTH',
  'start': 34512,
  'end': 39227},
 8: {'title': 'THE SECOND PART OF KING HENRY THE FOURTH',
  'start': 39228,
  'end': -100},
 9: {'title': 'THE LIFE OF KING HENRY V', 'start': -99, 'end': 47572},
 10: {'title': 'THE FIRST PART OF HENRY THE SIXTH',
  'start': 47573,
  'end': 50843},
 11: {'title': 'THE SECOND PART OF KING HENRY THE SIXTH',
  'start': 50844,
  'end': 54331},
 12: {'title': 'THE THIRD PART OF KING HENRY THE 

In [12]:
for e, i in enumerate(data):
    
    if "ALL’S WELL THAT ENDS WELL" in i:
        print(e)

2777


In [13]:
data[0]

'THE SONNETS'

In [14]:
# divide b/w plays and sonets
sonets = data[:2776]
plays = data[2777:]

In [15]:
data[0]

'THE SONNETS'

In [16]:
def long_lines(lst_ln):
    clean = []
    
    for ln in lst_ln: 
        
        if len(ln) == 0:
            pass
        else:
            pct = len(ln.strip(' ')) / len(ln)

            if pct >= .5:
                clean.append(ln.lstrip())

    return clean

In [ ]:
# May Not be Needed
#sonets = long_lines(sonets)
#plays = long_lines(plays)

## Word Encoding

This is just a start, and is not complete yet. 

In [17]:
vocab = list(set("\r\n".join(plays).split()))
words = [line.split() for line in plays]

In [18]:
len(vocab)

75738

## Character Encoding

Using the technique shown in lecture. 

In [19]:
text = '\r\n'.join(sonets)

chars = list(set(text))

char_int = {c:i for i,c in enumerate(chars)}
int_char = {i:c for i,c in enumerate(chars)}

print(f"Our corpus contains {len(chars)} unique characters.")

Our corpus contains 73 unique characters.


In [20]:
# Create the Sequence Data

maxlen = 150
step = 1

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 40 characters long
next_chars = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_chars.append(encoded[i + maxlen])
    
print('sequences:', len(sequences))

sequences: 100978


In [21]:
import numpy as np

# Specify x & y

x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_chars[i]] = 1

In [22]:
x.shape

(100978, 150, 73)

In [23]:
# build the model: a single LSTM

model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(chars)), dropout=0.2))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='nadam')

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 256)               337920    
_________________________________________________________________
dense (Dense)                (None, 73)                18761     
Total params: 356,681
Trainable params: 356,681
Non-trainable params: 0
_________________________________________________________________


In [25]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [26]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)

model.fit(x, y,
          batch_size=1024,
          validation_split=.2,
          epochs=100,
          callbacks=[print_callback, 
                     #EarlyStopping(min_delta=.02, monitor='val_loss', patience=10),
                     tensorboard_callback])

Train on 80782 samples, validate on 20196 samples
Epoch 1/100
79872/80782 [============================>.] - ETA: 1s - loss: 3.1565
----- Generating text after Epoch: 0
----- Generating with seed: "ht think me some untutored youth,
Unlearned in the world’s false subtleties.
Thus vainly thinking that she thinks me young,
Although she knows my d"
ht think me some untutored youth,
Unlearned in the world’s false subtleties.
Thus vainly thinking that she thinks me young,
tlthough she knows my doghlsmeftsnlreeotb e mr iekrg7  o
pmee t wrslf  epamh Odtiiit a iatwwi thoout
syntvAT trt

e dwlnhlol t  h ar‘ete thinnts ur,i’hti j   or o oscdgiseme  eim  y
tnatehtBo re lie  i ftMogjes nasa ervefystt eUetd ikamsbshdrWgit ord
s c a  t  -e i,w ed e darka r 
tie qmeheprncekth w ehsngae
 sp d ort roI twsi  c s t ?a:heIn hf   wgmy)u,e  n
ihApTiessircreK e,rke bvdoOnot,. 
80782/80782 [==============================] - 198s 2ms/sample - loss: 3.1553 - val_loss: 3.0101
Epoch 2/100
79872/80782 [============

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN